In [ ]:
from cellfinder.core.main import main as cellfinder_run
import tifffile

signal_array = tifffile.imread("tests/data/integration/detection/structure_split_test/signal/signal0000.tif")
background_array = tifffile.imread("tests/data/integration/detection/structure_split_test/background/background0000.tif")

voxel_sizes = [5, 2, 2] # in microns
detected_cells = cellfinder_run(signal_array,background_array,voxel_sizes)

## Testing the input file dimensions

In [ ]:
signal_array = tifffile.imread("tests/data/integration/detection/structure_split_test/signal/signal0000.tif")
background_array = tifffile.imread("tests/data/integration/detection/structure_split_test/background/background0000.tif")

In [ ]:
print(f"Signal array shape: {signal_array.shape}")
print(f"Background array shape: {background_array.shape}")

## Check if API functionality is working correctly or not

In [ ]:
# Just test the imports and function signatures
from cellfinder.core.classify import classify, ClassificationParameters, DataParameters

# Create dummy configuration objects
classification_params = ClassificationParameters(
    batch_size=64,
    network_depth="50-layer"
)

data_params = DataParameters(
    voxel_sizes=(5, 2, 2),
    network_voxel_sizes=(5, 2, 2)
)

# Just print the objects to verify they're correctly created
print("ClassificationParameters created successfully:", classification_params.__dict__)
print("DataParameters created successfully:", data_params.__dict__)

print("API changes implemented successfully!")

## Creating 3d volume 

In [1]:
import os
import numpy as np
import tifffile
from pathlib import Path

# Define paths
signal_dir = Path("tests/data/integration/detection/structure_split_test/signal")
background_dir = Path("tests/data/integration/detection/structure_split_test/background")

# Get sorted lists of image files
signal_files = sorted([f for f in os.listdir(signal_dir) if f.endswith('.tif')])
background_files = sorted([f for f in os.listdir(background_dir) if f.endswith('.tif')])

print(f"Found {len(signal_files)} signal images and {len(background_files)} background images")

# Load the first image to get dimensions
first_signal = tifffile.imread(signal_dir / signal_files[0])
print(f"Single image shape: {first_signal.shape}")

# Create empty arrays to hold stacked images
signal_array = np.zeros((len(signal_files), *first_signal.shape), dtype=first_signal.dtype)
background_array = np.zeros((len(background_files), *first_signal.shape), dtype=first_signal.dtype)

# Load all images into the arrays
for i, fname in enumerate(signal_files):
    signal_array[i] = tifffile.imread(signal_dir / fname)
    
for i, fname in enumerate(background_files):
    background_array[i] = tifffile.imread(background_dir / fname)

print(f"3D signal array shape: {signal_array.shape}")
print(f"3D background array shape: {background_array.shape}")

Found 21 signal images and 21 background images
Single image shape: (740, 740)
3D signal array shape: (21, 740, 740)
3D background array shape: (21, 740, 740)


## Cell detection using documentation

In [2]:
from cellfinder.core.main import main as cellfinder_run
from brainglobe_utils.cells.cells import Cell

# Define voxel sizes (in microns)
voxel_sizes = [5, 2, 2]

# Run detection pipeline
detected_cells = cellfinder_run(signal_array, background_array, voxel_sizes)

print(f"Number of detected cells: {len(detected_cells)}")
if detected_cells:
    print(f"Example detected cell: x: {detected_cells[0].x}, y: {detected_cells[0].y}, z: {detected_cells[0].z}, type: {detected_cells[0].type}")

# Print cell type constants for reference
print(f"Cell type constants: CELL={Cell.CELL}, NO_CELL={Cell.NO_CELL}")

Splitting cell clusters: 100%|██████████| 1/1 [00:09<00:00,  9.14s/it]


Detection complete. Found 40 cells in 0:00:19.014207
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 221ms/step
Classification complete - all points done in : 0:00:01.679569
Number of detected cells: 3
Example detected cell: x: 699, y: 49, z: 10, type: 2
Cell type constants: CELL=2, NO_CELL=1


In [ ]:
from cellfinder.core.classify.classify import classify_with_params, ClassificationParameters, DataParameters
from brainglobe_utils.cells.cells import Cell

# Your signal_array and background_array are already loaded and have shapes:
# (21, 740, 740) based on your previous output

# Create dummy cells positioned at valid locations within the array
detected_cells = [
    Cell([370, 370, 10], Cell.CELL),
    Cell([370, 370, 11], Cell.CELL)
]

# Create configuration objects with the CORRECT CUBE SIZES
data_params = DataParameters(
    voxel_sizes=(5, 2, 2),
    network_voxel_sizes=(5, 2, 2),
    n_free_cpus=2
)

classification_params = ClassificationParameters(
    batch_size=2,
    cube_height=50,  # Match the expected model input size
    cube_width=50,   # Match the expected model input size
    cube_depth=20,   # Match the expected model input size 
    network_depth="50-layer"
)

# Call the function directly
classified_cells = classify_with_params(
    points=detected_cells,
    signal_array=signal_array,
    background_array=background_array,
    data_parameters=data_params,
    classification_parameters=classification_params,
    model_name="resnet50_tv"
)

print(f"Classification complete. Number of classified cells: {len(classified_cells)}")

ValueError: Input 0 of layer "functional_1" is incompatible with the layer: expected shape=(None, 50, 50, 20, 2), found shape=(1, 20, 20, 5, 2)

In [ ]:
import brainglobe_utils.cells.cells
print(brainglobe_utils.cells.cells.__file__)